# Exploratory Data Analysis - Air Pollution Example


### Goals

* Assess the quantity, quality of your data.
* Determine which features are relevant for the model
* Is there structure in the data that needs to be modeled?

In [ ]:
#| output: false
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# plotting libs
import matplotlib.pyplot as plt
import plotnine as p9

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=12),
        axis_title_x=p9.element_text(size=11),
        axis_title_y=p9.element_text(size=11),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)

## Dataset: Exposure to particulate matter less than 2.5 microns in diameter

Our goal is to build a model of ground-level air pollution, using measurements from ground monitors together
with satellite imaging data.
The dataset is taken from github repository: https://github.com/jgabry/bayes-vis-paper
which contain all materials for the paper **_Visualization in Bayesian workflow_**:    

Gabry, J. , Simpson, D. , Vehtari, A. , Betancourt, M. and Gelman, A. (2019),     
Visualization in Bayesian workflow. _J. R. Stat. Soc. A_, 182: 389-402. doi:10.1111/rssa.12378

* Published JRSS version: https://rss.onlinelibrary.wiley.com/doi/full/10.1111/rssa.12378
* arXiv preprint: https://arxiv.org/pdf/1709.01449.pdf (includes Supplementary Materials in appendix)

A local version of the raw data for this paper is in the `data` directory, named `GM_spdf.RData`.
This [RDATA file](https://github.com/jgabry/bayes-vis-paper/blob/master/bayes-vis.RData) contains a single R [SpatialPointsDataFrame](https://rdrr.io/cran/sp/man/SpatialPoints.html).

We have converted this to a [GeoJson](https://geojson.org) file, `data/air_spdf.geojson` which contains just the information needed for this analysis.

The dataset for this paper consists of measurements of ambient ${PM}_{2.5}$ collected from ground monitors
together with the corresponding measurement from high-resolution satellite data for that geo-location.
The data was collected accros 100 countries, which have been grouped into 7 categories by the World Health Organization (WHO).
Each observation consists of the following:

- 3-letter ISO country code
- location name
- a pair of latitude, longitude coordinates (in WGS84)
- ${PM}_{2.5}$ measurements from ground monitors, and log(ground PM)
- ${PM}_{2.5}$ measurements from via high-resolution satellite data, and log(satelite PM)
- World Health Organization (WHO) super-region name
- 6-component hierarchical clustering of ground monitor measurements
- 6-component hierarchical clustering of log ground monitor measurements

In [ ]:
pm_gdf = gpd.read_file("data/air_spdf.geojson")
pm_gdf = pm_gdf.astype({col: 'category' for col in pm_gdf.select_dtypes(['int', 'object']).columns})
print(pm_gdf.info())

In [ ]:
pm_gdf[['iso3', 'log_pm25', 'log_sat', 'super_region_name', 'cluster_region', 'cluster_log_region']][:5]

To properly visualize the distribution of measurements around the globe we need a map which contains country boundaries.
We have downloaded a world map shapefile (in coordinate system WGS84) from
[Natural Earth](https://www.naturalearthdata.com/), which provides free public domain datasets.

We add the per-country WHO super region names and cluster regions to the world map so that we can color countries by super region and clustered region.
Because the air pollution dataset doesn't cover all countries, we create categories for "NaN" values and set them accordingly.

In [ ]:
world_map_gdf = gpd.read_file("data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
countries = pm_gdf[['iso3','super_region_name','cluster_region']].drop_duplicates(subset=['iso3'])
world_map_plus_gdf = world_map_gdf.merge(countries, left_on='ISO_A3', right_on='iso3', how='left')
# add categories for unknown countries, set "NaN" values accordingly
world_map_plus_gdf['cluster_region'] = world_map_plus_gdf['cluster_region'].cat.add_categories([7])
world_map_plus_gdf['cluster_region'] = world_map_plus_gdf[['cluster_region']].fillna(7)
world_map_plus_gdf['super_region_name'] = world_map_plus_gdf['super_region_name'].cat.add_categories(['NA'])
world_map_plus_gdf[['super_region_name']] = world_map_plus_gdf[['super_region_name']].fillna('NA')


We use this map and plot the locations of the ground monitors in this dataset, colored by super region.

In [ ]:
(p9.ggplot()
 + p9.geom_map(data=world_map_plus_gdf, mapping=p9.aes(fill='super_region_name'), alpha=0.2)
 + p9.geom_point(data=pm_gdf,
                 mapping=p9.aes(x='geometry.x', y='geometry.y', color='factor(super_region_name)'), size=1)
 + p9.scale_fill_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan', 'ivory'])
 + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan', 'ivory'])
 + p9.guides(fill=False)
 + p9.labs(color='Super Region')
 + p9.theme(figure_size=(20, 10), 
            axis_text_x=p9.element_blank(), axis_text_y=p9.element_blank(),
            axis_title_x=p9.element_blank(), axis_title_y=p9.element_blank())
)

Alternatively, can plot the level of $PM_{25}$ at each monitor location.  We use the midpoint of the values to visualize pollution levels from low (green) to high (red).

In [ ]:
pm_gdf[['log_pm25', 'log_sat']].describe()

In [ ]:
(p9.ggplot()
 + p9.geom_map(data=world_map_plus_gdf, mapping=p9.aes(fill='super_region_name'), alpha=0.2)
 + p9.geom_point(data=pm_gdf,
                 mapping=p9.aes(x='geometry.x', y='geometry.y', color='log_pm25'),
                 size=1)
 + p9.scale_color_gradient2(low='green', mid='yellow', high='red', midpoint=2.8)
 + p9.scale_fill_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan', 'ivory'])
 + p9.theme(figure_size=(20, 10), 
            axis_text_x=p9.element_blank(), axis_text_y=p9.element_blank(),
            axis_title_x=p9.element_blank(), axis_title_y=p9.element_blank())
)

## Data structure

The data has two levels of grouping:  low-level grouping by country (107 countries), and high-level grouping by super-region (7 categories) or cluster-region (6 clusters).

### Ground monitors per region

In [ ]:
(p9.ggplot()
 + p9.geom_bar(data=pm_gdf,
                 mapping=p9.aes(x='factor(super_region_name)', fill='factor(super_region_name)'), alpha=0.8)
 + p9.scale_fill_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan'])
 + p9.theme(figure_size=(8, 4), axis_text_x=p9.element_blank(),
            axis_title_x=p9.element_blank(), axis_title_y=p9.element_blank())
 + p9.ggtitle('Ground Monitors per Super Region') + p9.labs(fill='Super Region')
)

### Ground monitors per country

The number of per-country ground monitors depends on the size, overall development, and population density of the country.  China, the US, and India dominate, followed by other high income regions.

In [ ]:
monitors_per_country = pm_gdf.groupby('iso3').size().sort_values(ascending=False)
monitors_per_country.head(5)

In [ ]:
pm_gdf[['iso3']].value_counts().describe()

Of these 107 countries surveyed, the median number of monitors per country is 9.  We can visualize this distribution as a bar plot.  Is this useful?

In [ ]:
(p9.ggplot()
 + p9.geom_bar(data = pm_gdf,
                 mapping=p9.aes(x='factor(iso3)', fill='factor(super_region_name)'), alpha=0.8)
 + p9.scale_fill_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan'])
 + p9.theme(figure_size=(9, 4), axis_text_x=p9.element_blank())
 + p9.ggtitle('Ground Monitors per Country') + p9.xlab('country') + p9.labs(fill='Super Region')
)

## Plotting the relationship between ground monitors and satellite data

The goal of our analysis is to use the satellite data to predict the ground-level air pollution.  On the whole, the satellite measurements are lower than the ground monitors.

In [ ]:
pm_gdf[['log_pm25', 'log_sat']].describe()

We plot the log measurements overlaid with a simple linear regression showing the 95% central interval.
The outcome variable of interest is ground-level air pollution, therefore we plot this on the y-axis.

In [ ]:
# Plot all the data, color by super_region, pool all data for regression line
(p9.ggplot(pm_gdf, p9.aes(x='log_sat', y='log_pm25', color='factor(super_region_name)'))
    + p9.geom_point(alpha=0.5)
    + p9.geom_smooth(method='lm', color='grey', se=True)
    + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan'])
    + p9.xlab('log satellite')
    + p9.ylab('log ground monitor')
    + p9.theme(figure_size=(10, 6))
    + p9.ggtitle('satellite vs ground $PM_{2.5}$ readings, pooled regression') + p9.labs(color='Super Region')
)

In [ ]:
# Plot all the data, color by super_region, per-group regression lines

(p9.ggplot(pm_gdf, p9.aes(x='log_sat', y='log_pm25', color='factor(super_region_name)', group='factor(super_region_name)'))
    + p9.geom_point(alpha=0.5)
    + p9.geom_smooth(method='lm', se=True)
    + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan'])
    + p9.xlab('log satellite')
    + p9.ylab('log ground monitor')
    + p9.theme(figure_size=(10, 6))
    + p9.ggtitle('satellite vs ground $PM_{2.5}$ readings, super region regression') + p9.labs(color='Super Region')
)

In [ ]:
(p9.ggplot(pm_gdf, p9.aes(x='log_sat', y='log_pm25', color='factor(super_region_name)', group='factor(super_region_name)'))
    + p9.geom_smooth(method='lm', se=True)
    + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan'])
    + p9.xlab('log Satellite 2014')
    + p9.ylab('log PM 2.5')
    + p9.ggtitle('satellite vs ground $PM_{2.5}$ super region regressions') + p9.labs(color='Super Region')
    + p9.theme(figure_size=(8, 4))
)

As an alternative to the WHO super-regions, 
Gabry et al create a 6-component hierarchical clustering of ground monitor measurements of $PM_{2.5}$

In [ ]:
pm_gdf.groupby('cluster_region')[['log_pm25']].describe()

The clustering algorithm creates a few small regions.

In [ ]:
(p9.ggplot()
 + p9.geom_bar(data=pm_gdf,
            mapping=p9.aes(x='cluster_region', fill='cluster_region'), alpha=0.8)
 + p9.theme(figure_size=(6, 4), axis_title_x=p9.element_blank(), axis_title_y=p9.element_blank(),
            axis_text_x=p9.element_blank())
 + p9.scale_fill_manual(['red', 'orange', 'magenta', 'green', 'purple', 'cyan', 'ivory'])
 + p9.ggtitle('Ground Monitors per Cluster Region')
)

In [ ]:
(p9.ggplot()
 + p9.geom_map(data=world_map_plus_gdf, mapping=p9.aes(fill='cluster_region'), alpha=0.4)
 + p9.geom_point(data=pm_gdf,
                 mapping=p9.aes(x='geometry.x', y='geometry.y', color='factor(cluster_region)'), size=1)
 + p9.scale_fill_manual(['red', 'orange', 'magenta', 'green', 'purple', 'cyan', 'ivory'])
 + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'cyan', 'ivory'])
 + p9.guides(fill=False, color=False)
 + p9.theme(figure_size=(20, 10), 
            axis_text_x=p9.element_blank(), axis_text_y=p9.element_blank(),
            axis_title_x=p9.element_blank(), axis_title_y=p9.element_blank())
)

In [ ]:
(p9.ggplot(pm_gdf, p9.aes(x='log_sat', y='log_pm25', color='cluster_region', group='cluster_region'))
    + p9.geom_smooth(method='lm', se=True)
    + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'cyan', 'ivory'])
    + p9.xlab('log Satellite 2014')
    + p9.ylab('log PM 2.5')
    + p9.ggtitle('satellite vs ground $PM_{2.5}$ cluster region regressions') + p9.labs(color='Cluster region')
    + p9.theme(figure_size=(5, 4))
)

In [ ]:
(p9.ggplot(pm_gdf, p9.aes(x='log_sat', y='log_pm25', color='factor(super_region_name)', group='factor(super_region_name)'))
    + p9.geom_smooth(method='lm', se=True)
    + p9.scale_color_manual(['red', 'orange', 'magenta', 'green', 'purple', 'blue', 'cyan'])
    + p9.xlab('log Satellite 2014')
    + p9.ylab('log PM 2.5')
    + p9.ggtitle('satellite vs ground $PM_{2.5}$ super region regressions') + p9.labs(color='Super Region')
    + p9.theme(figure_size=(8, 4))
)